<a href="https://colab.research.google.com/github/mostafaGwely/sequence_generator_lstm/blob/master/model_co.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 


In [0]:
# !git clone https://github.com/mostafaGwely/sequence_generator_lstm

In [0]:
!ls 
df = pd.read_csv('./sequence_generator_lstm/neural_network_patent_query.csv')


In [0]:
df = df[:600]
df = df.patent_abstract

In [0]:
len(df)

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
T = Tokenizer()

In [0]:
T.fit_on_texts(df.values)

In [0]:
sequences = T.texts_to_sequences(df.values)

In [0]:
maxlen = 20

In [0]:
features = [] 
labels = [] 

In [0]:
for sequence in sequences:
    for i in range(maxlen,len(sequence)+1):
        extract = sequence[i-maxlen: i+1]
        features.append(extract[:-1])
        labels.append(extract[-1])

In [0]:
import numpy as np 

In [0]:
temp = np.zeros((len(features),  maxlen), dtype='int')

In [0]:
for i,d in enumerate(features):
    for j,dd in enumerate(d):
        temp[i][j] = dd

In [0]:
temp[0] 

In [0]:
features[0]

In [0]:
features = temp 

In [0]:
del temp

In [0]:
features[0] , labels[0]

In [0]:
idx_word = {}
for word, ind in T.word_index.items():
    idx_word[ind] = word


In [0]:
for i in features[8]:
    print(idx_word[i])
print(idx_word[labels[8]])

In [0]:
len(labels)
temp = np.zeros((len(labels), 1+len(idx_word)))
for i,d in enumerate(labels):
    temp[i][d] = 1


In [0]:
labels = temp
del temp
del df 

In [0]:

from keras.models import Sequential
from keras.layers import Dense, Embedding, Masking, LSTM, Dropout

In [0]:
num_words = 1+ len(idx_word)
num_words

In [27]:
# model = Sequential()
# model.add(Embedding(input_dim=num_words,input_length = maxlen,output_dim=100))
# model.add(LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_words, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.models import load_model
model = load_model('model.h5')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [0]:
x_train.shape , len(y_train[0])


In [0]:
model.summary()

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Create callbacks
# EarlyStopping(monitor='val_loss', patience=5),
callbacks = [
             ModelCheckpoint('model.h5', save_best_only=True, 
                             save_weights_only=False)]

In [0]:
history = model.fit(x_train, y_train, epochs=1000, validation_data=(x_test,y_test),batch_size=2500,
                    callbacks=callbacks)

In [0]:
model.predict(x_test[0:1,:])

In [0]:
x_test[0:1,:].shape

In [0]:
def idx_word_seq(l,idx_word = idx_word):
    idx_word[0] = ''
    a = [] 
    for i in l:
        a.append(idx_word[i])
    return a 

In [0]:
trigger = idx_word_seq(x_test[0:1,:].tolist()[0])

In [0]:
trigger, len(trigger)

In [0]:
trigger = np.array(list(T.texts_to_sequences(trigger))).reshape(1,-1)
trigger.shape

In [0]:
trigger.shape

In [0]:
conclusion = trigger


In [0]:
conclusion.shape

In [0]:
for i in range(100):
    x = model.predict(trigger)
    x = np.argsort(x)[0][:3]

    x = np.random.choice(x,1)[0]
    conclusion = np.append(conclusion,x).reshape(1,-1)
    trigger = conclusion[0][-20:].reshape(1,-1)

In [0]:
a = ''
for x in idx_word_seq(conclusion[0]):
    a += x
    a += ' '
a


In [82]:
!ls

model.h5			   sample_data
neural_network_patent_query.csv    sequence_generator_lstm
neural_network_patent_query.csv.1


In [0]:


from google.colab import files
files.upload()

Saving model.h5 to model.h5


In [0]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(T, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    T = pickle.load(handle)